In [52]:
import sqlite3
import csv
from datetime import datetime
def convertir_fecha(fecha):
    #Convertimos la fecha a date
    aux = datetime.strptime(fecha, '%d/%m/%Y %H:%M')
    #Convertimos la fecha al formato yyyy-mm-dd
    date = aux.strftime('%Y-%m-%d %H:%M')
    return date

def crear_bd(db_filename):
    '''
    '''
    conn = sqlite3.connect(db_filename)
    conn.execute("DROP TABLE IF EXISTS datos_generales")
    conn.execute("CREATE TABLE datos_generales (ticker TEXT PRIMARY KEY, nombre TEXT, indice TEXT, pais TEXT)")
    conn.execute("DROP TABLE IF EXISTS semanales_IBEX35")
    conn.execute("CREATE TABLE semanales_IBEX35 (ticker TEXT, fecha TEXT, precio REAL, FOREIGN KEY (ticker) REFERENCES datos_generales(ticker))")
    conn.commit()
    conn.close()

crear_bd("mercado.sqlite3")


def cargar_bd(db_filename, tab_datos, tab_ibex35):
    '''
    '''
    with open(tab_datos, 'r', newline='', encoding='utf8') as fich:
        datos_generales = csv.reader(fich, delimiter=';')
        lista_datos_generales = list(datos_generales)
    with open(tab_ibex35, 'r', newline='', encoding='utf8') as fich:
        datos_ibex = csv.reader(fich, delimiter=';')
        lista_datos_ibex = list(datos_ibex)
    conn = sqlite3.connect(db_filename)
    tabla_ibex = lista_datos_ibex[1::]
    for fila in tabla_ibex:
        fecha_original = fila[1]
        new_date = convertir_fecha(fecha_original)
        fila[1] = new_date

    conn.executemany("INSERT INTO datos_generales(ticker,nombre,indice,pais) VALUES (?,?,?,?)",lista_datos_generales[1:])
    conn.executemany("INSERT INTO semanales_IBEX35 (ticker, fecha, precio) VALUES (?, ?, ?)",tabla_ibex)
    # conn.execute("INSERT INTO semanales_IBEX35(ticker, fecha,precio) VALUES (?,?,?)",("falla","buenos dias",6))
    conn.commit()
    conn.close()




cargar_bd("mercado.sqlite3", "Tabla1.csv", "Tabla2.csv")

In [53]:
def consulta1(db_filename, indice):
    '''
    '''
    conn=sqlite3.connect(db_filename)
    cur=conn.execute("SELECT ticker, nombre FROM datos_generales WHERE indice = ? ORDER BY ticker ASC",[indice])
    return cur.fetchall()
    
print(consulta1("mercado.sqlite3","Nasdaq 100"))

[('AAPL', 'APPLE INC'), ('ABNB', 'AIRBNB RG-A'), ('ADBE', 'ADOBE SYSTEMS INC'), ('ADI', 'ANALOG DEVICES'), ('ADP', 'AUTOMATIC DATA PROC.'), ('ADSK', 'AUTODESK INC'), ('AEP', 'AMER.ELECT.POWER'), ('ALGN', 'ALIGN TECHNOLOGY'), ('AMAT', 'APPLIED MATERIALS'), ('AMD', 'ADVANCED MICRO DE'), ('AMGN', 'AMGEN INC'), ('AMZN', 'AMAZON.COM.INC'), ('ANSS', 'ANSYS'), ('ASML', 'ASML HOLDING'), ('ATVI', 'ACTIVISION BLIZZ'), ('AVGO', 'BROADCOM RG'), ('BIDU', 'BAIDU.COM'), ('BIIB', 'BIOGEN IDEC INC'), ('BKNG', 'BOOKING HOLDINGS'), ('CDNS', 'CADENCE DESIGN'), ('CHTR', 'CHARTER COMMUNICATIONS CL A'), ('CMCSA', 'COMCAST CO CL A'), ('COST', 'COSTCO WHOLESALE'), ('CPRT', 'COPART INC'), ('CRWD', 'CROWDSTRIKE HOLDINGS'), ('CSCO', 'CISCO SYSTEMS INC'), ('CSX', 'CSX CORP'), ('CTAS', 'CINTAS CORP'), ('CTSH', 'COGNIZANT TECH SOL'), ('DDOG', 'DATADOG RG-A'), ('DLTR', 'DOLLAR TREE'), ('DOCU', 'DOCUSIGN RG'), ('DXCM', 'DEXCOM'), ('EA', 'ELECTRONIC ARTS INC'), ('EBAY', 'EBAY.INC.'), ('EXC', 'EXELON'), ('FAST', 'FASTEN

In [54]:
def consulta2(db_filename):
    '''
    '''
    conn=sqlite3.connect(db_filename)
    cur=conn.execute('''SELECT G.ticker, G.nombre, MAX(S.precio)
                        FROM datos_generales G JOIN semanales_IBEX35 S ON G.ticker=S.ticker
                        GROUP BY G.ticker
                        ORDER BY G.nombre ASC''')
    return cur.fetchall()

print(consulta2("mercado.sqlite3"))


[('ANA', 'ACCIONA', 207.0), ('ACX', 'ACERINOX SA', 12.68), ('ACS', 'ACS', 26.6), ('AENA', 'AENA', 153.45), ('AMS', 'AMADEUS', 62.84), ('MTS', 'ARCELORMITTAL', 32.41), ('BBVA', 'BANCO DE BILBAO VIZCAYA', 6.13), ('SAB', 'BANCO DE SABADELL', 0.94), ('SAN', 'BANCO SANTANDER', 3.48), ('BKT', 'BANKINTER', 5.99), ('CABK', 'CAIXABANK', 3.53), ('CLNX', 'CELLNEX TELECOM', 54.62), ('ANE', 'CRP ACC ENER RN BR', 43.5), ('ENG', 'ENAGAS', 21.72), ('ELE', 'ENDESA', 20.86), ('FER', 'FERROVIAL', 27.56), ('FDR', 'FLUIDRA', 36.9), ('GRF', 'GRIFOLS CL.A', 20.5), ('IAG', 'IAG GROUP', 2.17), ('IBE', 'IBERDROLA', 11.31), ('ITX', 'INDITEX', 32.27), ('IDR', 'INDRA SISTEMAS', 10.95), ('COL', 'INMOBILIARIA COLONIAL', 8.68), ('ROVI', 'LABORATORIOS FARMACEUTICOS ROV', 73.8), ('MAP', 'MAPFRE', 2.01), ('MEL', 'MELIA HOTELS', 7.96), ('MRL', 'MERLIN PROPERTIES', 10.83), ('NTGY', 'NATURGY ENERGY', 29.58), ('PHM', 'PHARMA MAR', 76.42), ('RED', 'RED ELECTRICA', 19.63), ('REP', 'REPSOL', 15.49), ('SCYR', 'SACYR VALLEHERMOS

In [55]:
def consulta3(db_filename, limite):
    '''
    '''
    conn=sqlite3.connect(db_filename)
    cur=conn.execute('''SELECT G.ticker, G.nombre, AVG(S.precio) AS promedio, MAX(S.precio)-MIN(S.precio) AS diff
                        FROM datos_generales G JOIN semanales_IBEX35 S ON G.ticker=S.ticker
                        GROUP BY G.ticker
                        HAVING promedio > ?
                        ORDER BY promedio DESC
                     ''',[limite])
    return cur.fetchall()

print(consulta3("mercado.sqlite3",10))

[('ANA', 'ACCIONA', 173.75098039215692, 67.69999999999999), ('AENA', 'AENA', 135.57058823529408, 46.54999999999998), ('PHM', 'PHARMA MAR', 62.54470588235293, 25.060000000000002), ('ROVI', 'LABORATORIOS FARMACEUTICOS ROV', 60.5770588235294, 32.08), ('AMS', 'AMADEUS', 56.90039215686273, 16.470000000000006), ('CLNX', 'CELLNEX TELECOM', 43.47176470588235, 22.83), ('ANE', 'CRP ACC ENER RN BR', 34.240980392156864, 16.64), ('NTGY', 'NATURGY ENERGY', 26.57078431372548, 7.549999999999997), ('MTS', 'ARCELORMITTAL', 26.38568627450981, 11.779999999999998), ('FDR', 'FLUIDRA', 25.349803921568625, 22.14), ('FER', 'FERROVIAL', 25.210588235294114, 4.259999999999998), ('ITX', 'INDITEX', 24.468823529411758, 12.170000000000002), ('ACS', 'ACS', 23.245294117647067, 5.580000000000002), ('ENG', 'ENAGAS', 19.752352941176472, 5.879999999999999), ('ELE', 'ENDESA', 18.98470588235294, 5.4399999999999995), ('SLR', 'SOLARIA ENERGIA Y MEDIO AMBIEN', 18.891568627450976, 9.98), ('RED', 'RED ELECTRICA', 18.3141176470588

In [67]:

def consulta4(db_filename, ticker):
    '''
    '''
    conn=sqlite3.connect(db_filename)
    cur=conn.execute('''SELECT ticker,strftime('%Y-%m-%d', fecha) AS dia, precio
                        FROM semanales_IBEX35 
                        WHERE ticker=?
                        ORDER BY dia DESC
                     ''',[ticker])
    print(cur.fetchall())
consulta4('mercado.sqlite3', 'ACS') 

[('ACS', '2022-09-30', 23.13), ('ACS', '2022-09-23', 22.13), ('ACS', '2022-09-16', 22.92), ('ACS', '2022-09-09', 23.01), ('ACS', '2022-09-02', 22.62), ('ACS', '2022-08-26', 22.42), ('ACS', '2022-08-19', 23.88), ('ACS', '2022-08-12', 23.98), ('ACS', '2022-08-05', 23.14), ('ACS', '2022-07-29', 23.41), ('ACS', '2022-07-22', 22.22), ('ACS', '2022-07-15', 21.61), ('ACS', '2022-07-08', 21.69), ('ACS', '2022-07-01', 21.59), ('ACS', '2022-06-24', 23.8), ('ACS', '2022-06-17', 24.49), ('ACS', '2022-06-10', 25.32), ('ACS', '2022-06-03', 26.53), ('ACS', '2022-05-27', 26.6), ('ACS', '2022-05-20', 25.43), ('ACS', '2022-05-13', 24.31), ('ACS', '2022-05-06', 24.22), ('ACS', '2022-04-29', 24.59), ('ACS', '2022-04-22', 24.56), ('ACS', '2022-04-14', 24.91), ('ACS', '2022-04-08', 24.07), ('ACS', '2022-04-01', 24.59), ('ACS', '2022-03-25', 24.62), ('ACS', '2022-03-18', 24.1), ('ACS', '2022-03-11', 23.34), ('ACS', '2022-03-04', 21.27), ('ACS', '2022-02-25', 21.84), ('ACS', '2022-02-18', 22.32), ('ACS', '202